# CAE-LSTM Reduced Order Model

## Environment Setup

This notebook requires **MindSpore version >= 2.0.0** to support new APIs including: *mindspore.jit, mindspore.jit_class, mindspore.data_sink*. Please check [MindSpore Installation](https://www.mindspore.cn/install/en) for details.

In addition, **MindFlow version >=0.1.0** is also required. If it has not been installed in your environment, please select the right version and hardware, then install it as follows.

In [ ]:
mindflow_version = "0.1.0"  # update if needed
# GPU Comment out the following code if you are using NPU.
!pip uninstall -y mindflow-gpu
!pip install mindflow-gpu==$mindflow_version

# NPU Uncomment if needed.
# !pip uninstall -y mindflow-ascend
# !pip install mindflow-ascend==$mindflow_version

## Introduction

In order to effectively reduce the design cost and cycle time of using CFD methods, the reduced-order model (ROM) has gained wide attention in recent years. For complex compressible flows, using linear methods such as Proper Orthogonal Decomposition (POD) for flow field dimensionality reduction requires a large number of modes to ensure the reconstruction accuracy. It has been shown that the modes number can be effectively reduced by using nonlinear dimensionality reduction methods. Convolutional Autoencoder (CAE) is a kind of neural network composed of encoder and decoder, which can realize data dimensionality reduction and recon-struction, and can be regarded as a nonlinear extension of POD method. CAE is used for nonlinear dimension-ality reduction, and Long Short-Term Memory (LSTM) is used for time evolution. The CAE-LSTM can obtain high reconstruction and prediction accuracy on the premise of using less latents for unsteady compressible flows.

## Framework of CAE-LSTM

The CAE-LSTM reduced order model uses a CAE network to reduce the dimensionality of the flow field, extract the characteristics of the flow data, compress it into the hidden space of the encoder, and then use the LSTM network to perform coefficient time evolution on the free variables in the hidden space to obtain the free variables at other times of flow. Then, the decoder of the CAE network decodes the evolved free variables and reconstructs the flow field flow data at the corresponding time. The construction of the CAE-LSTM flow reduction model relies on the data reduction of the CAE network and the coefficient time evolution of the LSTM network. Compared with existing methods such as POD/DMD, using CAE networks for nonlinear dimensionality reduction of flow field data and LSTM networks for equation free evolution of free variables can achieve higher compression ratios and improve the efficiency of flow field prediction while ensuring the accuracy of the flow field reduction model.

+ Input：Input the flow field for a period of time.
+ Compression：Extract high-dimensional spatiotemporal flow characteristics by dimensionality reduction of the flow field using the encoder of CAE.
+ Evolution：Learning the evolution of spatiotemporal characteristics of low dimensional spatial flow fields through LSTM and predicting the next moment.
+ Reconstruction：Restore the predicted low-dimensional features of the flow field to high-dimensional space through the decoder of CAE.
+ Output：Output the predicted results of the transient flow field at the next moment.

The first step is to train the CAE network. After the training is completed, the CAE encoder is used to obtain the low dimensional features of the flow field. This low dimensional feature is used as the dataset of the LSTM network for LSTM network training.

![CAE-LSTM.png](./images/cae_lstm.png)

## Training environment

Import the required function library for training, where `src` includes dataset creation functions, network models and training loss visualization functions.

You can choose different cases to run, i.e. `sod`, `shu_osher`, `riemann`, `kh` or `cylinder`, among which `sod` and `shu_osher` are one-dimension cases, and `riemann`, `kh` and `cylinder` are two-dimension cases. You can change the case name in the `case` of `parser.add_argument` to run the corresponding case. And if you use the command line to run network training, you can also write the case name after `--case` to run the corresponding case. Default `sod`.

The static GRAPH of Mindspore framework is adopted for training. Training can be done on GPU (default) or Ascend (single card).

In [1]:
import os
import time
import argparse

import numpy as np

from mindspore import nn, ops, context, save_checkpoint, set_seed, jit, data_sink
from mindflow.utils import load_yaml_config
from src import create_cae_dataset, CaeNet1D, CaeNet2D, plot_train_loss

In [2]:
np.random.seed(0)
set_seed(0)

In [3]:
parser = argparse.ArgumentParser(description='CaeNet')
parser.add_argument("--case", type=str, default="sod", choices=["sod", "shu_osher", "riemann", "kh"],
                    help="Which case to run, support 'sod', 'shu_osher', 'riemann', 'kh'")
parser.add_argument("--mode", type=str, default="GRAPH", choices=["GRAPH", "PYNATIVE"],
                    help="Context mode, support 'GRAPH', 'PYNATIVE'")
parser.add_argument("--save_graphs", type=bool, default=False, choices=[True, False],
                    help="Whether to save intermediate compilation graphs")
parser.add_argument("--save_graphs_path", type=str, default="./graphs")
parser.add_argument("--device_target", type=str, default="GPU", choices=["GPU", "Ascend"],
                    help="The target device to run, support 'Ascend', 'GPU'")
parser.add_argument("--device_id", type=int, default=0, help="ID of the target device")
parser.add_argument("--config_file_path", type=str, default="./config.yaml")
args = parser.parse_args()

context.set_context(case=args.case,
                    mode=context.GRAPH_MODE if args.mode.upper().startswith("GRAPH") else context.PYNATIVE_MODE,
                    save_graphs=args.save_graphs,
                    save_graphs_path=args.save_graphs_path,
                    device_target=args.device_target,
                    device_id=args.device_id,
                    config_file_path=args.config_file_path)
use_ascend = context.get_context(attr_key='device_target') == "Ascend"

## CAE training parameter settings

Import corresponding parameter configurations for the dataset, CAE model, and optimizer from the config.yaml file according to the case chosen. As for the cylinder case, you should indicate the Reynolds number of the case to read the corresponding data for training.

In [4]:
config = load_yaml_config(args.config_file_path)
if args.case == 'sod' or args.case == 'shu_osher':
    data_params = config["1D_cae_data"]
    model_params = config["1D_cae_model"]
    optimizer_params = config["1D_cae_optimizer"]
else:
    data_params = config["2D_cae_data"]
    model_params = config["2D_cae_model"]
    optimizer_params = config["2D_cae_optimizer"]

The default path for saving loss files during training is optimizer_params ["summary_dir"], the weight parameters are saved in the ckpt folder.

In [5]:
summary_dir = optimizer_params["summary_dir"]
if not os.path.exists(summary_dir):
    os.mkdir(summary_dir)
ckpt_dir = os.path.join(summary_dir, 'ckpt')
if not os.path.exists(ckpt_dir):
    os.mkdir(ckpt_dir)

## Construct CAE neural network

The CAE network consists of multiple layers of convolution and maximum pooling to form an encoder, and multiple layers of convolution and upsampling to form a decoder. Use MSELoss loss function and Adam optimizer.

In [6]:
if args.case == 'sod' or args.case == 'shu_osher':
    cae = CaeNet1D(model_params["data_dimension"], model_params["conv_kernel_size"],
                   model_params["maxpool_kernel_size"], model_params["maxpool_stride"],
                   model_params["encoder_channels"], model_params["decoder_channels"])
else:
    cae = CaeNet2D(model_params["data_dimension"], model_params["conv_kernel_size"],
                   model_params["maxpool_kernel_size"], model_params["maxpool_stride"],
                   model_params["encoder_channels"], model_params["decoder_channels"],
                   model_params["channels_dense"])

loss_fn = nn.MSELoss()
cae_opt = nn.Adam(cae.trainable_params(), optimizer_params["lr"], weight_decay=optimizer_params["weight_decay"])

## CAE dataset

Dataset download address: [data_driven/cae-lstm/dataset](https://download.mindspore.cn/mindscience/mindflow/dataset/applications/data_driven/cae-lstm)

After importing the dataset, perform data sinking settings.

In [7]:
cae_dataset, _ = create_cae_dataset(data_params["data_path"], data_params["batch_size"], data_params["multiple"])

sink_process = data_sink(train_step, cae_dataset, sink_size=1)
train_data_size = cae_dataset.get_dataset_size()

## CAE training

Build forward_fn and train_step, start training the CAE network and visualize the training loss.

In [8]:
def forward_fn(data, label):
    logits = cae(data)
    loss = loss_fn(logits, label)
    return loss

grad_fn = ops.value_and_grad(forward_fn, None, cae_opt.parameters, has_aux=False)

@jit
def train_step(data, label):
    loss, grads = grad_fn(data, label)
    loss = ops.depend(loss, cae_opt(grads))
    return loss

print(f"====================Start CaeNet train=======================")
train_loss = []
for epoch in range(1, optimizer_params["epochs"] + 1):
    local_time_beg = time.time()
    cae.set_train()
    epoch_train_loss = 0
    for _ in range(train_data_size):
        epoch_train_loss = ops.squeeze(sink_process(), axis=())
    train_loss.append(epoch_train_loss)
    print(f"epoch: {epoch} train loss: {epoch_train_loss} epoch time: {time.time() - local_time_beg:.2f}s")

    if epoch % optimizer_params["save_ckpt_interval"] == 0:
        save_checkpoint(cae, f"{ckpt_dir}/cae_{epoch}.ckpt")
print(f"=====================End CaeNet train========================")
plot_train_loss(train_loss, summary_dir, optimizer_params["epochs"], "cae")

pid:23104
====================Start CaeNet train=======================
epoch: 1 train loss: 0.00859989 epoch time: 3.23s
epoch: 2 train loss: 0.00563688 epoch time: 0.52s
epoch: 3 train loss: 0.00485115 epoch time: 0.53s
epoch: 4 train loss: 0.00341164 epoch time: 0.62s
epoch: 5 train loss: 0.00332990 epoch time: 0.57s
...
epoch: 4396 train loss: 3.69731242e-06 epoch time: 0.51s
epoch: 4397 train loss: 2.65247831e-06 epoch time: 0.55s
epoch: 4398 train loss: 1.14417275e-06 epoch time: 0.54s
epoch: 4399 train loss: 4.97764995e-06 epoch time:0.52s
epoch: 4400 train loss: 2.48092419e-06 epoch time: 0.55s
====================End CaeNet train=======================


## CAE flow field reconstruction results

After training the CAE network, run `cae_eval.py` to view the training results of CAE to determine whether to continue training the LSTM network

The following figures show the real flow field, CAE flow field reconstruction results, and the error curves between them in the five cases. The first two flow field results show the variation in the flow field over time, while the third error curve shows the average error of the CAE reconstructed flow field and the real flow field label over time. The errors meeting the accuracy requirements for flow field reconstruction.

Sod shock tube:

<figure class="harf">
    <img src="./images/sod_cae_reconstruction.gif" title="sod_cae_reconstruction" width="600"/>
    <img src="./images/sod_cae_error.png" title="sod_cae_error" width="300"/>
</center>

Shu_Osher problem:

<figure class="harf">
    <img src="./images/shu_osher_cae_reconstruction.gif" title="shu_osher_cae_reconstruction" width="600"/>
    <img src="./images/shu_osher_cae_error.png" title="shu_osher_cae_error" width="300"/>
</center>

Riemann problem:

<figure class="harf">
    <img src="./images/riemann_cae_reconstruction.gif" title="riemann_cae_reconstruction" width="600"/>
    <img src="./images/riemann_cae_error.png" title="riemann_cae_error" width="300"/>
</center>

Kelvin-Helmholtz instability problem:

<figure class="harf">
    <img src="./images/kh_cae_reconstruction.gif" title="kh_cae_reconstruction" width="600"/>
    <img src="./images/kh_cae_error.png" title="kh_cae_error" width="300"/>
</center>

cylinder flow (Re = 200):

<figure class="harf">
    <img src="./images/cylinder_cae_reconstruction.gif" title="cylinder_cae_reconstruction" width="600"/>
    <img src="./images/cylinder_cae_error.png" title="cylinder_cae_error" width="300"/>
</center>

## LSTM framework and training Settings

The construction of LSTM network framework, training environment, and other related processing are similar to those of CAE network.

Firstly, import the required function library for training, then import the LSTM network dataset setting parameters, LSTM model, and optimizer parameter settings. The default training loss save path is optimizer_params ["summary_dir"], the weight parameters are saved in the ckpt folder. The network consists of multiple LSTM layers and a full connection layer, using MSELoss loss function and Adam optimizer.

In [9]:
import os
import time
import argparse

import numpy as np

from mindspore import nn, ops, context, save_checkpoint, set_seed, jit, data_sink
from mindflow.utils import load_yaml_config
from src import create_lstm_dataset, Lstm, plot_train_loss
from cae_eval import cae_eval

np.random.seed(0)
set_seed(0)

parser = argparse.ArgumentParser(description='Lstm')
parser.add_argument("--case", type=str, default="sod", choices=["sod", "shu_osher", "riemann", "kh"],
                    help="Which case to run, support 'sod', 'shu_osher', 'riemann', 'kh'")
parser.add_argument("--mode", type=str, default="GRAPH", choices=["GRAPH", "PYNATIVE"],
                    help="Context mode, support 'GRAPH', 'PYNATIVE'")
parser.add_argument("--save_graphs", type=bool, default=False, choices=[True, False],
                    help="Whether to save intermediate compilation graphs")
parser.add_argument("--save_graphs_path", type=str, default="./graphs")
parser.add_argument("--device_target", type=str, default="GPU", choices=["GPU", "Ascend"],
                    help="The target device to run, support 'Ascend', 'GPU'")
parser.add_argument("--device_id", type=int, default=0, help="ID of the target device")
parser.add_argument("--config_file_path", type=str, default="./config.yaml")
args = parser.parse_args()

context.set_context(case=args.case,
                    mode=context.GRAPH_MODE if args.mode.upper().startswith("GRAPH") else context.PYNATIVE_MODE,
                    save_graphs=args.save_graphs,
                    save_graphs_path=args.save_graphs_path,
                    device_target=args.device_target,
                    device_id=args.device_id,
                    config_file_path=args.config_file_path)
use_ascend = context.get_context(attr_key='device_target') == "Ascend"

# prepare params
config = load_yaml_config(args.config_file_path)
if args.case == 'sod' or args.case == 'shu_osher':
    data_params = config["1D_lstm_data"]
    model_params = config["1D_lstm_model"]
    optimizer_params = config["1D_lstm_optimizer"]
else:
    data_params = config["1D_lstm_data"]
    model_params = config["1D_lstm_model"]
    optimizer_params = config["1D_lstm_optimizer"]

# prepare summary file
summary_dir = optimizer_params["summary_dir"]
ckpt_dir = os.path.join(summary_dir, 'ckpt')

# prepare model
lstm = Lstm(model_params["latent_size"], model_params["hidden_size"], model_params["num_layers"])
loss_fn = nn.MSELoss()
lstm_opt = nn.Adam(lstm.trainable_params(), optimizer_params["lr"], weight_decay=optimizer_params["weight_decay"])

## LSTM dataset loading and processing

The LSTM network dataset is obtained by the CAE encoder, and data sinking settings are performed after creating the dataset.

In [10]:
# prepare dataset
latent_true = cae_eval(args.config_file_path, args.case)
lstm_dataset, _ = create_lstm_dataset(latent_true, data_params["batch_size"], data_params["time_size"],
                                      data_params["latent_size"], data_params["time_window"],
                                      data_params["gaussian_filter_sigma"])

# data sink
sink_process = data_sink(train_step, lstm_dataset, sink_size=1)
train_data_size = lstm_dataset.get_dataset_size()

## LSTM training

Build forward_fn and train_step, start training the LSTM network and visualize the training loss.

In [11]:
# Define forward function
def forward_fn(data, label):
    logits = lstm(data)
    loss = loss_fn(logits, label)
    return loss

# Get gradient function
grad_fn = ops.value_and_grad(forward_fn, None, lstm_opt.parameters, has_aux=False)

@jit
def train_step(data, label):
    loss, grads = grad_fn(data, label)
    loss = ops.depend(loss, lstm_opt(grads))
    return loss

print(f"====================Start Lstm train=======================")
train_loss = []
for epoch in range(1, optimizer_params["epochs"] + 1):
    local_time_beg = time.time()
    lstm.set_train()
    epoch_train_loss = 0
    for _ in range(train_data_size):
        epoch_train_loss = ops.squeeze(sink_process(), axis=())
    train_loss.append(epoch_train_loss)
    print(f"epoch: {epoch} train loss: {epoch_train_loss} epoch time: {time.time() - local_time_beg:.2f}s")

    if epoch % optimizer_params["save_ckpt_interval"] == 0:
        save_checkpoint(lstm, f"{ckpt_dir}/lstm_{epoch}.ckpt")
print(f"=====================End Lstm train========================")
plot_train_loss(train_loss, summary_dir, optimizer_params["epochs"], "lstm")

pid:22152
====================Start Lstm train=======================
epoch: 1 train loss: 0.4425844 epoch time: 3.75s
epoch: 2 train loss: 0.23611887 epoch time: 0.75s
epoch: 3 train loss: 0.65945524 epoch time: 0.76s
epoch: 4 train loss: 0.77271056 epoch time: 0.80s
epoch: 5 train loss: 0.3535387 epoch time: 0.81s
...
epoch: 4396 train loss: 9.665465e-05 epoch time: 0.76s
epoch: 4397 train loss: 5.5045904e-05 epoch time: 0.77s
epoch: 4398 train loss: 0.00013155791 epoch time: 0.80s
epoch: 4399 train loss: 0.0769522e-05 epoch time: 0.80s
epoch: 4400 train loss: 0.0010389996 epoch time: 0.80s
====================End Lstm train=======================


## Visualization of predicted flow field results

Run `cae_lstm_eval.py` to view the prediction results of the CAE-LSTM reduced order model

The following figures show the actual flow field, the predicted results of the CAE-LSTM network, and the corresponding average error in the five cases. The overall prediction time errors meeting the accuracy requirements of flow field prediction.

Sod shock tube:

<figure class="harf">
    <img src="./images/sod_cae_lstm_predict.gif" title="sod_cae_lstm_predict" width="600"/>
    <img src="./images/sod_cae_lstm_error.png" title="sod_cae_lstm_error" width="300"/>
</center>

Shu_Osher problem:

<figure class="harf">
    <img src="./images/shu_osher_cae_lstm_predict.gif" title="shu_osher_cae_lstm_predict" width="600"/>
    <img src="./images/shu_osher_cae_lstm_error.png" title="shu_osher_cae_lstm_error" width="300"/>
</center>

Riemann problem:

<figure class="harf">
    <img src="./images/riemann_cae_lstm_predict.gif" title="riemann_cae_lstm_predict" width="600"/>
    <img src="./images/riemann_cae_lstm_error.png" title="riemann_cae_lstm_error" width="300"/>
</center>

Kelvin-Helmholtz instability problem:

<figure class="harf">
    <img src="./images/kh_cae_lstm_predict.gif" title="kh_cae_lstm_predict" width="600"/>
    <img src="./images/kh_cae_lstm_error.png" title="kh_cae_lstm_error" width="300"/>
</center>

cylinder flow (Re = 200):

<figure class="harf">
    <img src="./images/cylinder_cae_lstm_predict.gif" title="cylinder_cae_lstm_predict" width="600"/>
    <img src="./images/cylinder_cae_lstm_error.png" title="cylinder_cae_lstm_error" width="300"/>
</center>